In [3]:
import json
import httpx
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO


In [4]:
url = 'https://www.sofascore.com/api/v1/player/934386/image'

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Referer': 'https://www.sofascore.com/',
    'Origin': 'https://www.sofascore.com'
}

with httpx.Client(headers=headers) as client:
    response = client.get(url)
    i = Image.open(BytesIO(response.content))



If it's a large request, use tqdm to provide user with download time:

```
import tempfile

import httpx
from tqdm import tqdm

with tempfile.NamedTemporaryFile() as download_file:
    url = "https://speed.hetzner.de/100MB.bin"
    with httpx.stream("GET", url) as response:
        total = int(response.headers["Content-Length"])

        with tqdm(total=total, unit_scale=True, unit_divisor=1024, unit="B") as progress:
            num_bytes_downloaded = response.num_bytes_downloaded
            for chunk in response.iter_bytes():
                download_file.write(chunk)
                progress.update(response.num_bytes_downloaded - num_bytes_downloaded)
                num_bytes_downloaded = response.num_bytes_downloaded
```

Note: Don't literally code this, use it as a reference. It's just opening a temp file then running a request to website


Making multiple requests at once:

```
import asyncio
import httpx

async def do_tasks():
    async with httpx.AsyncClient() as client:
        tasks = [client.get(f"http://my-api/{url_param}") for url_param in parameters]
        result = await asyncio.gather(*tasks)

```

In [7]:
url = 'https://www.sofascore.com/api/v1/event/11352568/shotmap'

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Referer': 'https://www.sofascore.com/',
    'Origin': 'https://www.sofascore.com'
}

with httpx.Client(headers=headers) as client:
    response = client.get(url)
    data = response.json().get('shotmap')


In [36]:
pd.json_normalize(data).time.unique()

array([ 0, 95, 93, 92, 91, 90, 88, 87, 85, 83, 82, 81, 80, 79, 78, 77, 76,
       75, 74, 73, 72, 71, 70, 69, 67, 66, 63, 62, 61, 59, 58, 56, 55, 54,
       53, 52, 50, 48, 47, 46, 45, 49, 44, 42, 41, 40, 39, 37, 36, 35, 34,
       33, 32, 31, 29, 28, 26, 24, 23, 22, 21, 18, 17, 16, 15, 14, 13, 12,
       11,  7,  6,  2])

In [45]:
cool = pd.json_normalize(data)

cool['time_min'] = (cool['timeSeconds'] / 60).apply(np.floor)

cool.time_min.unique()

array([93., 88., 85., 83., 81., 80., 68., 66., 59., 56., 54., 52., 48.,
       47., 45., 44., 42., 41., 39., 35., 32., 31., 29., 22., 18., 14.,
       13.,  6.])

In [23]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15"
}

In [9]:
url = "https://www.sofascore.com/api/v1/event/11352568/comments"

response = requests.get(url, headers = headers)
lineups = response.json()

In [10]:
game = pd.json_normalize(lineups.get('comments'))

In [12]:
game

,text,type,id,time,periodName,isHome,player.name,player.firstName,player.lastName,player.slug,...,assist1.firstName,assist1.lastName,assist1.slug,assist1.shortName,assist1.position,assist1.jerseyNumber,assist1.userCount,assist1.id,assist1.fieldTranslations.nameTranslation.ar,assist1.fieldTranslations.shortNameTranslation.ar
0,"Match ends, Luton Town 2, Fulham 4.",matchEnded,23924545,-2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Second Half ends, Luton Town 2, Fulham 4.",endSecondHalf,23924492,88,2ND,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Attempt missed. Bobby De Cordova-Reid (Fulham)...,shotOffTarget,23924481,93,2ND,True,Bobby Decordova-Reid,,,bobby-decordova-reid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Andros Townsend (Luton Town) wins a free kick ...,freeKickWon,23924436,94,2ND,False,Andros Townsend,NaN,NaN,andros-townsend,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Foul by Fodé Ballo-Touré (Fulham).,freeKickLost,23924435,94,2ND,False,Fodé Ballo-Touré,,,fode-ballo-toure,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,"Corner, Fulham. Conceded by Daiki Hashioka.",cornerKick,23921683,12,1ST,False,Daiki Hashioka,,,daiki-hashioka,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,Attempt blocked. João Palhinha (Fulham) right ...,shotBlocked,23921634,5,1ST,False,João Palhinha,NaN,NaN,joao-palhinha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,Foul by Elijah Adebayo (Luton Town).,freeKickLost,23921469,-1,1ST,False,Elijah Adebayo,NaN,NaN,adebayo-elijah,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,Tim Ream (Fulham) wins a free kick in the defe...,freeKickWon,23921468,0,1ST,False,Tim Ream,NaN,NaN,tim-ream,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
game['time'].unique()

array([-2, 88, 93, 94, 98, 81, 96, 97, 83, 79, 85, 90, 75, 73, 76, 84, 68,
       69, 78, 77, 80, 66, 67, 72, 71, 64, 65, 62, 57, 70, 59, 60, 48, 61,
       51, 56, 55, 45, 46, 58, 44, 54, 52, 47, 53, 41, 39, 42, 49, 34, 38,
       33, 29, 32, 30, 31, 35, 28, 27, 40, 26, 36, 24, 19, 22, 16, 12, 17,
       10,  6, 15,  8, 20, 21,  5, -1,  0, -6])

In [25]:
shots = pd.json_normalize(lineups.get('shotmap'))

In [5]:
shots.columns

Index(['isHome', 'shotType', 'situation', 'bodyPart', 'goalMouthLocation',
       'xg', 'id', 'time', 'addedTime', 'timeSeconds', 'reversedPeriodTime',
       'reversedPeriodTimeSeconds', 'incidentType', 'player.name',
       'player.firstName', 'player.lastName', 'player.slug',
       'player.shortName', 'player.position', 'player.jerseyNumber',
       'player.userCount', 'player.id',
       'player.fieldTranslations.nameTranslation.ar',
       'player.fieldTranslations.shortNameTranslation.ar',
       'playerCoordinates.x', 'playerCoordinates.y', 'playerCoordinates.z',
       'goalMouthCoordinates.x', 'goalMouthCoordinates.y',
       'goalMouthCoordinates.z', 'draw.start.x', 'draw.start.y', 'draw.end.x',
       'draw.end.y', 'draw.goal.x', 'draw.goal.y', 'xgot',
       'blockCoordinates.x', 'blockCoordinates.y', 'blockCoordinates.z',
       'draw.block.x', 'draw.block.y', 'goalType'],
      dtype='object')

In [26]:
shots[['time', 'timeSeconds', 'player.name']]

,time,timeSeconds,player.name
0,86,5632,Bobby Decordova-Reid
1,85,5325,Cauley Woodrow
2,81,5157,Alex Iwobi
3,86,5127,Bobby Decordova-Reid
4,88,5000,Bobby Decordova-Reid
5,72,4911,Tom Cairney
6,89,4844,Carlton Morris
7,79,4821,Tahith Chong
8,71,4079,Harry Wilson
9,59,3969,Chiedozie Ogbene


In [109]:
game.head(50)

,text,type,id,time,periodName,isHome,player.name,player.firstName,player.lastName,player.slug,...,assist1.firstName,assist1.lastName,assist1.slug,assist1.shortName,assist1.position,assist1.jerseyNumber,assist1.userCount,assist1.id,assist1.fieldTranslations.nameTranslation.ar,assist1.fieldTranslations.shortNameTranslation.ar
0,"Match ends, Luton Town 2, Fulham 4.",matchEnded,23924545,-7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Second Half ends, Luton Town 2, Fulham 4.",endSecondHalf,23924492,89,2ND,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Attempt missed. Bobby De Cordova-Reid (Fulham)...,shotOffTarget,23924481,94,2ND,False,Bobby Decordova-Reid,,,bobby-decordova-reid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Andros Townsend (Luton Town) wins a free kick ...,freeKickWon,23924436,93,2ND,True,Andros Townsend,NaN,NaN,andros-townsend,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Foul by Fodé Ballo-Touré (Fulham).,freeKickLost,23924435,93,2ND,True,Fodé Ballo-Touré,,,fode-ballo-toure,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Foul by Daiki Hashioka (Luton Town).,freeKickLost,23924438,95,2ND,True,Daiki Hashioka,,,daiki-hashioka,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Bobby De Cordova-Reid (Fulham) wins a free kic...,freeKickWon,23924437,81,2ND,True,Bobby Decordova-Reid,,,bobby-decordova-reid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Pelly Ruddock Mpanzu (Luton Town) wins a free ...,freeKickWon,23924440,89,2ND,False,Pelly Ruddock Mpanzu,NaN,NaN,pelly-ruddock-mpanzu,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Foul by Fodé Ballo-Touré (Fulham).,freeKickLost,23924439,96,2ND,False,Fodé Ballo-Touré,,,fode-ballo-toure,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Substitution, Fulham. Timothy Castagne replace...",substitution,23924441,88,2ND,True,Timothy Castagne,,,timothy-castagne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
shots['time'].unique()

array([97, 94, 78, 91, 77, 71, 64, 68, 65, 53, 54, 60, 43, 41, 45, 35, 47,
       38, 27, 36, 39, 26, 28, 14,  6])

In [11]:
game.columns

Index(['text', 'type', 'id', 'time', 'periodName', 'isHome', 'player.name',
       'player.slug', 'player.shortName', 'player.position',
       'player.jerseyNumber', 'player.userCount', 'player.id',
       'player.fieldTranslations.nameTranslation.ar',
       'player.fieldTranslations.shortNameTranslation.ar', 'player.firstName',
       'player.lastName', 'assist1.name', 'assist1.slug', 'assist1.shortName',
       'assist1.position', 'assist1.jerseyNumber', 'assist1.userCount',
       'assist1.id', 'assist1.fieldTranslations.nameTranslation.ar',
       'assist1.fieldTranslations.shortNameTranslation.ar', 'playerIn.name',
       'playerIn.slug', 'playerIn.shortName', 'playerIn.position',
       'playerIn.jerseyNumber', 'playerIn.userCount', 'playerIn.id',
       'playerOut.name', 'playerOut.slug', 'playerOut.shortName',
       'playerOut.position', 'playerOut.jerseyNumber', 'playerOut.userCount',
       'playerOut.id', 'playerOut.fieldTranslations.nameTranslation.ar',
       'playerOut

In [98]:
BASE_URL = 'https://sofascore.com/api/v1/event'

stats = requests.get((BASE_URL + '/11352406/statistics')).json()
event = requests.get((BASE_URL + '/11352406')).json()

In [99]:
df = pd.json_normalize(stats.get('statistics')[0].get('groups'), record_path=['statisticsItems']) # ALL period
df.set_index("name", inplace=True)
df = df.T.iloc[:2, :].reset_index()

In [102]:
cool = pd.json_normalize(event)

In [110]:
home = cool['event.homeTeam.name']
away = cool['event.awayTeam.name']

In [111]:
df

name,index,Ball possession,Expected goals,Big chances,Total shots,Goalkeeper saves,Corner kicks,Fouls,Passes,Tackles,...,Aerial duels,Dribbles,Tackles won,Total tackles,Interceptions,Recoveries,Clearances,Total saves,Goals prevented,Goal kicks
0,home,74%,4.32,5,37,1,12,7,763,20,...,9/23 (39%),10/18 (56%),65%,20,5,52,3,1,0.23,1
1,away,26%,0.51,2,4,9,1,6,276,13,...,14/23 (61%),7/11 (64%),54%,13,8,44,28,9,-1.18,16


In [114]:
df['team'] = np.where(df['index'] == 'home', home, away)

In [115]:
df

name,index,Ball possession,Expected goals,Big chances,Total shots,Goalkeeper saves,Corner kicks,Fouls,Passes,Tackles,...,Dribbles,Tackles won,Total tackles,Interceptions,Recoveries,Clearances,Total saves,Goals prevented,Goal kicks,team
0,home,74%,4.32,5,37,1,12,7,763,20,...,10/18 (56%),65%,20,5,52,3,1,0.23,1,Manchester City
1,away,26%,0.51,2,4,9,1,6,276,13,...,7/11 (64%),54%,13,8,44,28,9,-1.18,16,Luton Town


In [101]:
list(pd.json_normalize(event).columns)

['event.tournament.name',
 'event.tournament.slug',
 'event.tournament.category.name',
 'event.tournament.category.slug',
 'event.tournament.category.sport.name',
 'event.tournament.category.sport.slug',
 'event.tournament.category.sport.id',
 'event.tournament.category.id',
 'event.tournament.category.country.alpha2',
 'event.tournament.category.country.alpha3',
 'event.tournament.category.country.name',
 'event.tournament.category.flag',
 'event.tournament.category.alpha2',
 'event.tournament.uniqueTournament.name',
 'event.tournament.uniqueTournament.slug',
 'event.tournament.uniqueTournament.primaryColorHex',
 'event.tournament.uniqueTournament.secondaryColorHex',
 'event.tournament.uniqueTournament.category.name',
 'event.tournament.uniqueTournament.category.slug',
 'event.tournament.uniqueTournament.category.sport.name',
 'event.tournament.uniqueTournament.category.sport.slug',
 'event.tournament.uniqueTournament.category.sport.id',
 'event.tournament.uniqueTournament.category.id

In [53]:
stats = pd.json_normalize(df.explode('statisticsItems')['statisticsItems'])

In [80]:
stats.head()

,name,home,away,compareCode,statisticsType,valueType,homeValue,awayValue,renderType,key,homeTotal,awayTotal
0,Ball possession,74%,26%,1,positive,event,74.00,26.00,2,ballPossession,NaN,NaN
1,Expected goals,4.32,0.51,1,positive,event,4.32,0.51,1,expectedGoals,NaN,NaN
2,Big chances,5,2,1,positive,event,5.00,2.00,1,bigChanceCreated,NaN,NaN
3,Total shots,37,4,1,positive,event,37.00,4.00,1,totalShotsOnGoal,NaN,NaN
4,Goalkeeper saves,1,9,2,positive,event,1.00,9.00,1,goalkeeperSaves,NaN,NaN


In [85]:
stats.set_index("name", inplace=True)



KeyError: "None of ['name'] are in the columns"

In [90]:
# Transpose the DataFrame
stats.T.iloc[:2, :].reset_index()

name,index,Ball possession,Expected goals,Big chances,Total shots,Goalkeeper saves,Corner kicks,Fouls,Passes,Tackles,...,Aerial duels,Dribbles,Tackles won,Total tackles,Interceptions,Recoveries,Clearances,Total saves,Goals prevented,Goal kicks
0,home,74%,4.32,5,37,1,12,7,763,20,...,9/23 (39%),10/18 (56%),65%,20,5,52,3,1,0.23,1
1,away,26%,0.51,2,4,9,1,6,276,13,...,14/23 (61%),7/11 (64%),54%,13,8,44,28,9,-1.18,16


In [19]:
import json

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager

In [20]:
options = webdriver.ChromeOptions()
options.set_capability(
    "goog:loggingPrefs", {"performance": "ALL", "browser": "ALL"}
)


driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_page_load_timeout(10)

try:
    driver.get("https://www.sofascore.com/luton-town-fulham/Tsxb#id:11352568")
except:
    pass


driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [21]:
logs_raw = driver.get_log("performance")
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

In [22]:
for x in logs:
    path = x['params'].get('headers', {}).get(':path', '') # Extract the ':path' value from the headers, defaulting to an empty string if not found
    if '/api/' in path:
        print(path)

/api/v1/team/43/image
/api/v1/team/72/image
/api/v1/event/11352568/statistics
/api/v1/event/11352568/lineups
/api/v1/tournament/1/season/52186/standings/total
/api/v1/event/11352568/comments
/api/v1/event/11352568/graph/win-probability
/api/v1/country/alpha2
/api/v1/sport/-14400/event-count
/api/v1/event/11352568/incidents
/api/v1/event/11352568/votes
/api/v1/event/11352568/pregame-form
/api/v1/event/11352568/featured-players
/api/v1/event/11352568
/api/v1/event/11352568/comments
/api/v1/event/11352568/highlights
/api/v1/tv/event/11352568/country-channels
/api/v1/event/11352568/team-streaks
/api/v1/event/Tsxb/h2h/events
/api/v1/team/72/events/last/0
/api/v1/team/72/events/next/0
/api/v1/team/43/events/last/0
/api/v1/team/43/events/next/0
/api/201540/envelope/?sentry_key=d693747a6bb242d9bb9cf7069fb57988&sentry_version=7&sentry_client=sentry.javascript.nextjs%2F7.113.0
/api/v1/event/11352568/win-probability
/api/v1/event/11352568/graph
/api/v1/player/901892/attribute-overviews
/api/v1/pl

In [29]:
import requests

response = requests.get('https://www.sofascore.com/api/v1/event/11352568/shotmap').json()



In [30]:
response

{'shotmap': [{'player': {'name': 'Bobby Decordova-Reid',
    'firstName': '',
    'lastName': '',
    'slug': 'bobby-decordova-reid',
    'shortName': 'B. Reid',
    'position': 'M',
    'jerseyNumber': '14',
    'userCount': 984,
    'id': 151499,
    'fieldTranslations': {'nameTranslation': {'ar': 'بوبي رايد'},
     'shortNameTranslation': {'ar': 'ب. رايد'}}},
   'isHome': False,
   'shotType': 'miss',
   'situation': 'assisted',
   'playerCoordinates': {'x': 0.46312342279836116, 'y': 30, 'z': 0},
   'bodyPart': 'right-foot',
   'goalMouthLocation': 'close-right',
   'goalMouthCoordinates': {'x': 0, 'y': 43.7544765865799, 'z': 18.1},
   'xg': 0.037010721862316,
   'id': 3261646,
   'time': 91,
   'addedTime': 4,
   'timeSeconds': 5632,
   'draw': {'start': {'x': 30, 'y': 18.671578597331646},
    'end': {'x': 75.36682046019887, 'y': 0},
    'goal': {'x': 68.27508758105684, 'y': 65.57111213486027}},
   'reversedPeriodTime': 1,
   'reversedPeriodTimeSeconds': 668,
   'incidentType': 'sh